In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import kagglehub
from PIL import Image
import torchsummary

# Download latest version
path = kagglehub.dataset_download("yudhaislamisulistya/plants-type-datasets")

print("Path to dataset files:", path)
import os
os.listdir(path)

Path to dataset files: C:\Users\PC\.cache\kagglehub\datasets\yudhaislamisulistya\plants-type-datasets\versions\16


['split_ttv_dataset_type_of_plants']

In [53]:
ROOT_PATH = os.path.join(path,"split_ttv_dataset_type_of_plants")
os.listdir(ROOT_PATH)

['Test_Set_Folder', 'Train_Set_Folder', 'Validation_Set_Folder']

In [54]:
train_dir = os.path.join(ROOT_PATH,"Train_Set_Folder")
test_dir = os.path.join(ROOT_PATH,"Test_Set_Folder")
val_dir = os.path.join(ROOT_PATH,"Validation_Set_Folder")

In [ ]:
transform = transforms.Compose([
    transforms.Resize(size=(300,300)),
    transforms.ToTensor()
])

In [56]:
train_data = ImageFolder(root=train_dir,transform=transform)
test_data = ImageFolder(root=test_dir,transform=transform)
val_data = ImageFolder(root=val_dir,transform=transform)

In [57]:
train_dataloader = DataLoader(train_data,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=32,shuffle=False)
val_dataloader = DataLoader(val_data,batch_size=32,shuffle=False)

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  

def train_step(model, train_loader, loss_fn, optimizer, device):
    model.train()  
    train_loss, train_acc = 0, 0
    
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  
        outputs = model(images)  
        loss = loss_fn(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        _, preds = torch.max(outputs, 1)
        train_acc += (preds == labels).sum().item()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_acc /= len(train_loader.dataset)
    return train_loss, train_acc


def val_step(model, val_loader, loss_fn, device):
    model.eval()  
    val_loss, val_acc = 0, 0

    with torch.no_grad():  # Gradient hesaplama kapalı
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_acc += (preds == labels).sum().item()
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_acc /= len(val_loader.dataset)
    return val_loss, val_acc


In [ ]:

class MobileNetV2Transfer(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = torchvision.models.mobilenet_v2(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

In [60]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_mobile = MobileNetV2Transfer(num_classes=30).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_mobile.parameters(), lr=0.001)
EPOCHS = 15
for epoch in range(EPOCHS):
    train_loss, train_acc = train_step(model_mobile, train_dataloader, loss_fn, optimizer, device)
    val_loss, val_acc = val_step(model_mobile, val_dataloader, loss_fn, device)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

c:\Users\PC\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Validating: 100%|██████████| 95/95 [00:11<00:00,  8.09it/s]


Epoch 1/15
Train Loss: 0.9953, Train Acc: 0.7171
Val Loss: 0.5778, Val Acc: 0.8205


Validating: 100%|██████████| 95/95 [00:11<00:00,  8.47it/s]


Epoch 2/15
Train Loss: 0.6102, Train Acc: 0.8086
Val Loss: 0.4634, Val Acc: 0.8455


Validating: 100%|██████████| 95/95 [00:11<00:00,  8.47it/s]


...

Epoch 13/15
Train Loss: 0.4080, Train Acc: 0.8645
Val Loss: 0.3155, Val Acc: 0.8980


Validating: 100%|██████████| 95/95 [00:11<00:00,  7.94it/s]


Epoch 14/15
Train Loss: 0.4142, Train Acc: 0.8640
Val Loss: 0.3149, Val Acc: 0.8888


Validating: 100%|██████████| 95/95 [00:11<00:00,  8.35it/s]

Epoch 15/15
Train Loss: 0.3996, Train Acc: 0.8682
Val Loss: 0.3163, Val Acc: 0.8911


In [61]:
model_mobile.eval()  
test_loss, test_acc = 0, 0

with torch.no_grad():  # Gradient hesaplama kapalı
    for images, labels in tqdm(test_dataloader, desc="Validating"):
        images, labels = images.to(device), labels.to(device)

        outputs = model_mobile(images)
        loss = loss_fn(outputs, labels)

        _, preds = torch.max(outputs, 1)
        test_acc += (preds == labels).sum().item()
        test_loss += loss.item()

test_loss /= len(test_dataloader)
test_acc /= len(test_dataloader.dataset)
print(test_acc,test_loss)

Validating: 100%|██████████| 94/94 [00:11<00:00,  8.52it/s]

0.8925950633755837 0.35376690347936557


In [62]:
torchsummary.summary(model_mobile,input_size=(3,300,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]             864
       BatchNorm2d-2         [-1, 32, 150, 150]              64
             ReLU6-3         [-1, 32, 150, 150]               0
            Conv2d-4         [-1, 32, 150, 150]             288
       BatchNorm2d-5         [-1, 32, 150, 150]              64
             ReLU6-6         [-1, 32, 150, 150]               0
            Conv2d-7         [-1, 16, 150, 150]             512
       BatchNorm2d-8         [-1, 16, 150, 150]              32
  InvertedResidual-9         [-1, 16, 150, 150]               0
           Conv2d-10         [-1, 96, 150, 150]           1,536
      BatchNorm2d-11         [-1, 96, 150, 150]             192
            ReLU6-12         [-1, 96, 150, 150]               0
           Conv2d-13           [-1, 96, 75, 75]             864
      BatchNorm2d-14           [-1, 96,

In [63]:
torch.save(model_mobile.state_dict(),"models/mobel_mobile15epochs.pt")

In [64]:
model_deneme = MobileNetV2Transfer(30).to(device)
model_deneme.load_state_dict(torch.load("models/mobel_mobile15epochs.pt"))

<All keys matched successfully>

In [65]:
model_deneme.eval()

scripted_model = torch.jit.script(model_deneme) 
scripted_model.save("models/model_native.pt")  